In [4]:
# A changer in the arithmetic context precision may cause x to differ from +x
import decimal

ctx = decimal.getcontext()
ctx.prec = 40
one_third = decimal.Decimal('1') / decimal.Decimal('3')
one_third

Decimal('0.3333333333333333333333333333333333333333')

In [5]:
one_third == +one_third

True

In [6]:
ctx.prec = 28
one_third == +one_third

False

In [9]:
# false because +one_third produces a new instance
+one_third

Decimal('0.3333333333333333333333333333')

In [10]:
one_third

Decimal('0.3333333333333333333333333333333333333333')

In [13]:
from collections import Counter

ct = Counter('abdbracadabra')
ct

Counter({'a': 5, 'b': 3, 'd': 2, 'r': 2, 'c': 1})

In [16]:
ct['r'] = -3

In [17]:
ct['d'] = 0

In [18]:
ct

Counter({'a': 5, 'b': 3, 'c': 1, 'd': 0, 'r': -3})

In [20]:
+ct # NOTICE! this excludes `-` and 0!

Counter({'a': 5, 'b': 3, 'c': 1})

In [81]:
# class from Chapter12 (some methods added)
from array import array
import reprlib
import operator
import math
import functools
import itertools
from collections.abc import Sized, Iterable

class Vector:
    __match_args__ = ('x', 'y', 'z', 't')
    typecode = 'd'
    
    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)  # used to limit the console logs with ...
        components = components[components.find('['):-1]
        return f'Vector({components})'

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))

    def __eq__(self, other):
        if isinstance(other, Vector):
            return (len(self) == len(other) and
                    all(a == b for a, b in zip(self, other)))
        else:
            return NotImplemented
        # 3rd iteration
        # return len(self) == len(other) and all(a == b for a, b in zip(self, other))
        # 2nd iteration
        # if len(self) != len(other):
        #     return False
        # for a, b in zip(self, other):
        #     if a != b:
        #         return False
        # return True
        # 1st iteration
        # return tuple(self) == tuple(other) # creating two tuples, and what about large multidimensional vectors

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

    def __len__(self):
        return len(self._components)

    def __getitem__(self, key):
        if isinstance(key, slice):
            cls = type(self)
            return cls(self._components[key])
        index = operator.index(key)
        return self._components[index]

    def __getattr__(self, name):
        cls = type(self)
        try:
            pos = cls.__match_args__.index(name)
        except ValueError:
            pos = -1
        if 0 <= pos < len(self._components):
            return self._components[pos]
        msg = f'{cls.__name_!r} object has no attribute {name!r}'
        raise AttributeError(msg)

    def __setattr__(self, name, value):
        cls = type(self)
        if len(name) == 1:
            if name in cls.__match_args__:
                error = 'readonly attribute {attr_name!r}'
            elif name.islower():
                error = "can't set attributes 'a' to 'z' in {cls_name!r}"
            else:
                error = ''
            if error:
                msg = error.format(cls_name=cls.__name__, attr_name=name)
                raise AttributeError(msg)
        super().__setattr__(name, value)

    def __hash__(self):
        hashes = (hash(x) for x in self._components)
        # hashes = map(hash, self._components) # Python 2 would be slower here but Python 3 map is lazy it creates a generator
        return functools.reduce(operator.xor, hashes, 0)

    def angle(self, n):
        r = math.hypot(*self[n:])
        a = math.atan2(r, self[n-1])
        if (n == len(self) - 1) and (self[-1] < 0):
            return math.pi * 2 - a
        else:
            return a

    def angles(self):
        return (self.angle(n) for n in range(1, len(self)))

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('h'): # hyperspherical coordinates
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)], self.angles())
            outer_fmt = '<{}>'
        else:
            coords = self
            outer_fmt = '({})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(', '.join(components))

    def __abs__(self):
        return math.hypot(*self)

    def __neg__(self):
        return Vector(-x for x in self)

    def __pos__(self):
        return Vector(self)

    # Overload `+`
    def __add__(self, other):
        try:
            pairs = itertools.zip_longest(self, other, fillvalue=0.0)
            return Vector(a + b for a, b in pairs)
        except TypeError:
            return NotImplemented

    # Overload `*`
    def __mul__(self, scalar):
        try:
            factor = float(scalar)
        except TypeError:
            return NotImplemented
        return Vector(n * scalar for n in self)

    def __rmul__(self, scalar):
        return self * scalar

    # or simply __radd__ = __add__
    def __radd__(self, other):
        return self + other

    # v1 @ v2 matrix multiply
    def __matmul__(self, other):
        if (isinstance(other, Sized) and
            isinstance(other, Iterable)):
            if len(self) == len(other):
                return sum(a * b for a, b in zip(self, other))
            else:
                raise ValueError('@ requires vectors of equal length.')
        else:
            return NotImplemented

    def __rmatmul__(self, other):
        return self @ other

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

In [35]:
v1 = Vector([3,4,5])
v2 = Vector([6,7,8])
v1 + v2

Vector([9.0, 11.0, 13.0])

In [36]:
v1 + v2 == Vector([3+6, 4+7, 5+8])

True

In [37]:
v1 = Vector([3,4,5,6])
v3 = Vector([1,2])
v1 + v3 # if the vectors are of different lengths we just implicitly fill with zeros

Vector([4.0, 6.0, 5.0, 6.0])

In [38]:
# class from Chapter 11
class Vector2d:
    __match_args__ = ('x', 'y') # you can see why this is needed below, we don't need to include all the public instance attributes in case of optional
    __slots__ = ('__x', '__y') # why? shown below
    typecode = 'd'

    # x, and y should be private hense two leading underscore, sometimes _ is more common but _ is sometimes used as protected and some people think protected means private :(
    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y)

    @property
    def x(self):
        return self.__x

    @property
    def y(self):
        return self.__y

    def __iter__(self):
        return (i for i in (self.x, self.y))

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

    # added later to work with format
    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'): # using p for polar coordinates
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)

    def angle(self):
        return math.atan2(self.x, self.y)

    def __hash__(self):
        return hash((self.x, self.y))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

In [39]:
v1 = Vector([3,4,5])
v1 + (10,20,30)

Vector([13.0, 24.0, 35.0])

In [40]:
v2d = Vector2d(1,2)
v1 + v2d

Vector([4.0, 6.0, 5.0])

In [54]:
v1 = Vector([3,4,5])
(10,20,30) + v1 # TypeError before adding __radd__

Vector([13.0, 24.0, 35.0])

In [55]:
v2d = Vector2d(1,2)
v2d + v1 # TypeError before adding __radd__

Vector([4.0, 6.0, 5.0])

In [50]:
# We can introduce a __radd__ (reverse add, which will invoke as a fallback)

In [51]:
v1 = Vector([1.0,2.0,3.0])
14 * v1

Vector([14.0, 28.0, 42.0])

In [53]:
v1 * True # Remember booleans are subclassed of int!

Vector([1.0, 2.0, 3.0])

In [56]:
from fractions import Fraction
v1 * Fraction(1,3)

Vector([1.0, 1.3333333333333333, 1.6666666666666665])

In [76]:
# a @ b where @ being a infix operator meaning the dot product of 2 matricies
# for the longest we had to use numpy.dot(a,b) but even NumPy adopted @
# added __matmul__ in vector for this

In [69]:
va = Vector([1,2,3])
vz = Vector([5,6,7])
va @ vz == 38.0 # 1*5 + 2*6 + 3*7

True

In [73]:
[10,30,30] @ vz

440.0

In [74]:
va @ 3

TypeError: unsupported operand type(s) for @: 'Vector' and 'int'

In [86]:
va = Vector([1.0,2.0,3.0])
vb = Vector(range(1,4))
va == vb

True

In [87]:
vc = Vector([1,2])
v2d = Vector2d(1,2)
vc == v2d

True

In [90]:
t3 = (1,2,3)
va == t3

False

In [97]:
v1 = Vector([1,2,3])
v1_alias = v1
id(v1)

4410641904

In [98]:
v1 += Vector([4,5,6])
v1

Vector([5.0, 7.0, 9.0])

In [99]:
id(v1)

4410642192

In [100]:
v1_alias

Vector([1.0, 2.0, 3.0])

In [101]:
v1 *= 11
v1

Vector([55.0, 77.0, 99.0])

In [102]:
id(v1)

4410654672

In [103]:
# if the inplace operators are not implemented they work as syntatic sugar:
# a += b becomes a = a + b, if we have __add__ then += will work with no additional code

In [105]:
# from chapter 13
import random
import abc

class Tombola(abc.ABC):
    
    @abc.abstractmethod
    def load(self, iterable):
        """Add itemss from an iterable."""

    @abc.abstractmethod
    def pick(self):
        """Remove item at random, returning it.
        This method should raise `LookupError` when the instance is empty."""

    def loaded(self):
        """Returen `True` if there's at least 1 item, `False`, otherwise."""
        return bool(self.inspect())

    def inspect(self):
        """Return a sorted tuple with the items currenlty inside."""
        items = []
        while True:
            try:
                items.append(self.pick())
            except LookupError: # LookupError -> IndexError or KeyError
                break;
        self.load(items)
        return tuple(items)


class BingoCage(Tombola):
    def __init__(self, items):
        self._randomizer = random.SystemRandom() # uses os.urandom() random bytes suitable for cryptographic use
        self._items = []
        self.load(items)

    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)

    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')

    def __call__(self):
        self.pick()

In [106]:
class AddableBingoCage(BingoCage):
    def __add__(self, other):
        if isinstance(other, Tombola):
            return AddableBingoCage(self.inspect() + other.inspect())
        else:
            return NotImplemented

    def __iadd__(self, other):
        if isinstance(other, Tombola):
            other_iterable = other.inspect()
        else:
            try:
                other_iterable = iter(other)
            except TypeError:
                msg = 'right operand in += must be "Tombola" or an iterable'
                raise TypeError(msg)
        self.load(other_iterable)
        return self

In [108]:
vowels = 'AEIOU'
globe = AddableBingoCage(vowels)
globe.inspect()

('O', 'A', 'I', 'E', 'U')

In [109]:
globe.pick() in vowels

True

In [110]:
len(globe.inspect())

4

In [111]:
globe2 = AddableBingoCage('XYZ')
globe3 = globe + globe2
len(globe3.inspect())

7

In [113]:
void = globe + [10,20] # unsupported operand

TypeError: unsupported operand type(s) for +: 'AddableBingoCage' and 'list'

In [114]:
globe_orig = globe
len(globe.inspect())

4

In [115]:
globe += globe2
len(globe.inspect())

7

In [116]:
globe += ['M','N']
len(globe.inspect())

9

In [117]:
globe is globe_orig

True

In [118]:
globe += 1 # noniterable

TypeError: right operand in += must be "Tombola" or an iterable

In [119]:
"""
my_list + x concatenates one list to another list, but my_list += x,
you can extend the lefthand list with items from any iterable x
on the righthand side. This is how the list.extend() method
works, it accepts any iterable argument.
"""

'\nmy_list + x concatenates one list to another list, but my_list += x,\nyou can extend the lefthand list with items from any iterable x\non the righthand side. This is how the list.extend() method\nworks, it accepts any iterable argument.\n'